# **Project InsightAds**  
### **Digital-Marketing-Performance-Analysis**

**Team Name** : Nova  
**Members** : 'Izzah Huwaidah (Ketua), Ashma Nisa Sholiha   
**Competition** : inSight Data – Data Analyst Competition  
**Date** : January - February 2026


## **Latar Belakang**

Periklanan digital merupakan salah satu strategi utama dalam meningkatkan 
brand awareness dan penjualan. Namun, peningkatan anggaran iklan tidak selalu 
diikuti dengan peningkatan performa yang sebanding. Oleh karena itu, diperlukan 
analisis data yang komprehensif untuk mengevaluasi efektivitas iklan, mengidentifikasi 
pola performa, serta menemukan peluang optimasi berbasis data.


## **Deskripsi Dataset**

Dataset yang digunakan berisi data performa iklan digital yang mencakup metrik 
seperti impressions, clicks, content views, add to cart, purchases, biaya iklan, 
serta nilai pembelian. Data ini digunakan untuk menganalisis efektivitas iklan 
berdasarkan berbagai objective kampanye dan periode waktu tertentu.


## **Impor Library**

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv("Data Ads - Kompetisi Data Analyts by inSight Data Batch 01.csv")

## **Data Cleaning and Preparation** (kalo emang perlu nanti)

In [3]:
df.head()

,created_date,account_name,campaign_objective,impressions,reach,clicks,link_clicks,content_views,add_to_cart,purchase,purchase_value,amount_spent
0,2023-01-01,Client A - Fashion,Traffic,915434.0,81436.0,92554.0,15544.0,0.0,0.0,0.0,0.0,2357742.0
1,2023-01-01,Client A - Fashion,Sales,103328.0,89200.0,22100.0,38038.0,1199.0,817.0,824.0,3353702.0,3578529.0
2,2023-01-01,Client B - Beauty,Traffic,336015.0,141082.0,26321.0,46107.0,0.0,0.0,0.0,0.0,1482711.0
3,2023-01-01,Client B - Beauty,Sales,864410.0,400596.0,10719.0,29397.0,3395.0,960.0,591.0,3470984.0,4335272.0
4,2023-01-01,Client C - Fashion,Traffic,916254.0,319415.0,32995.0,37496.0,0.0,0.0,0.0,0.0,1897353.0


In [4]:
df.shape

(4380, 12)

In [5]:
# checking for null values
df.isnull().sum()

created_date          0
account_name          0
campaign_objective    0
impressions           0
reach                 0
clicks                0
link_clicks           0
content_views         0
add_to_cart           0
purchase              0
purchase_value        0
amount_spent          0
dtype: int64

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4380 entries, 0 to 4379
Data columns (total 12 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   created_date        4380 non-null   object 
 1   account_name        4380 non-null   object 
 2   campaign_objective  4380 non-null   object 
 3   impressions         4380 non-null   float64
 4   reach               4380 non-null   float64
 5   clicks              4380 non-null   float64
 6   link_clicks         4380 non-null   float64
 7   content_views       4380 non-null   float64
 8   add_to_cart         4380 non-null   float64
 9   purchase            4380 non-null   float64
 10  purchase_value      4380 non-null   float64
 11  amount_spent        4380 non-null   float64
dtypes: float64(9), object(3)
memory usage: 410.8+ KB


In [7]:
df.columns

Index(['created_date', 'account_name', 'campaign_objective', 'impressions',
       'reach', 'clicks', 'link_clicks', 'content_views', 'add_to_cart',
       'purchase', 'purchase_value', 'amount_spent'],
      dtype='object')

In [11]:
# =========================
# 2) TRIM & NORMALISASI TEKS
# =========================
for col in ["account_name", "campaign_objective"]:
    df[col] = df[col].astype(str).str.strip()

# Normalisasi nilai objective (biar konsisten: Traffic/Sales)
df["campaign_objective"] = (
    df["campaign_objective"]
      .str.lower()
      .replace({
          "traffic ": "traffic",
          " sales": "sales",
          "conversion": "sales",
          "conversions": "sales"
      })
)

# Pakai title-case biar enak di laporan
df["campaign_objective"] = df["campaign_objective"].str.title()


In [12]:
# =========================
# 3) KONVERSI TIPE TANGGAL
# =========================
df["created_date"] = pd.to_datetime(df["created_date"], errors="coerce")

# Jika ada tanggal yang gagal diparse → cek
bad_date = df["created_date"].isna().sum()
print("Tanggal gagal parse:", bad_date)
# Kalau bad_date > 0, nanti kita inspeksi barisnya:
# display(df[df["created_date"].isna()].head())

Tanggal gagal parse: 0


In [13]:
# =========================
# 4) KONVERSI KOLOM ANGKA (SAFE COERCE)
# =========================
num_cols = [
    "impressions","reach","clicks","link_clicks","content_views",
    "add_to_cart","purchase","purchase_value","amount_spent"
]

for c in num_cols:
    # bersihin kalau ada pemisah ribuan koma, dll.
    df[c] = (
        df[c]
          .astype(str)
          .str.replace(",", "", regex=False)
          .str.replace(" ", "", regex=False)
    )
    df[c] = pd.to_numeric(df[c], errors="coerce")



In [14]:
# =========================
# 5) HANDLE MISSING VALUES (SIMPLE & MASUK AKAL)
# =========================
# Karena metrik ads umumnya "0 berarti tidak ada event", missing biasanya diperlakukan 0.
df[num_cols] = df[num_cols].fillna(0)

# Untuk text: jika ada kosong, isi "Unknown" biar tidak hilang
df["account_name"] = df["account_name"].replace({"nan": np.nan}).fillna("Unknown")
df["campaign_objective"] = df["campaign_objective"].replace({"Nan": np.nan, "nan": np.nan}).fillna("Unknown")



In [15]:
# =========================
# 6) DROP DUPLICATES (HATI-HATI)
# =========================
# Dataset kamu: "1 baris = 1 campaign per hari per klien" (berdasarkan guidebook)
# Tapi karena kita nggak punya kolom campaign_id/name, paling aman:
# - cek duplikat full-row dulu
before = df.shape[0]
df = df.drop_duplicates()
after = df.shape[0]
print(f"Duplikat full-row yang dibuang: {before - after}")



Duplikat full-row yang dibuang: 0


In [16]:
# =========================
# 7) BASIC VALIDATION / SANITY CHECK (TANPA OUTLIER DULU)
# =========================

# (a) Nilai negatif harusnya tidak ada
neg_counts = (df[num_cols] < 0).sum().sort_values(ascending=False)
print("\nJumlah nilai negatif per kolom:")
print(neg_counts[neg_counts > 0])

# (b) Cek logika funnel minimal (boleh ada edge-case, tapi ini buat flag aja)
# Catatan: clicks kadang bisa > impressions di sebagian platform kalau definisinya beda,
# tapi umumnya: impressions >= clicks >= link_clicks dan link_clicks >= purchase.
flags = pd.DataFrame({
    "clicks_gt_impressions": df["clicks"] > df["impressions"],
    "link_clicks_gt_clicks": df["link_clicks"] > df["clicks"],
    "purchase_gt_link_clicks": df["purchase"] > df["link_clicks"],
    "reach_gt_impressions": df["reach"] > df["impressions"],
    "purchase_value_positive_but_purchase_zero": (df["purchase_value"] > 0) & (df["purchase"] == 0),
    "spent_zero_but_has_activity": (df["amount_spent"] == 0) & ((df["impressions"] > 0) | (df["clicks"] > 0) | (df["purchase"] > 0)),
})

print("\nJumlah baris ter-flag (sanity):")
print(flags.sum().sort_values(ascending=False))

# Jika mau lihat contoh baris bermasalah:
# display(df[flags.any(axis=1)].head(20))




Jumlah nilai negatif per kolom:
Series([], dtype: int64)

Jumlah baris ter-flag (sanity):
link_clicks_gt_clicks                        870
reach_gt_impressions                         845
clicks_gt_impressions                          0
purchase_gt_link_clicks                        0
purchase_value_positive_but_purchase_zero      0
spent_zero_but_has_activity                    0
dtype: int64


In [17]:
# =========================
# 8) FEATURE PREP BUAT ANALISIS (OPSIONAL TAPI BERGUNA)
# =========================
# Tambah kolom bulan untuk trend
df["year"] = df["created_date"].dt.year
df["month"] = df["created_date"].dt.to_period("M").astype(str)

# Pastikan tipe purchase integer-like (kalau datamu float karena hasil baca)
for c in ["impressions","reach","clicks","link_clicks","content_views","add_to_cart","purchase"]:
    df[c] = df[c].round(0).astype(int)

print("\nShape akhir:", df.shape)
display(df.head())



Shape akhir: (4380, 14)


,created_date,account_name,campaign_objective,impressions,reach,clicks,link_clicks,content_views,add_to_cart,purchase,purchase_value,amount_spent,year,month
0,2023-01-01,Client A - Fashion,Traffic,915434,81436,92554,15544,0,0,0,0.0,2357742.0,2023,2023-01
1,2023-01-01,Client A - Fashion,Sales,103328,89200,22100,38038,1199,817,824,3353702.0,3578529.0,2023,2023-01
2,2023-01-01,Client B - Beauty,Traffic,336015,141082,26321,46107,0,0,0,0.0,1482711.0,2023,2023-01
3,2023-01-01,Client B - Beauty,Sales,864410,400596,10719,29397,3395,960,591,3470984.0,4335272.0,2023,2023-01
4,2023-01-01,Client C - Fashion,Traffic,916254,319415,32995,37496,0,0,0,0.0,1897353.0,2023,2023-01


## **Feature Engineering (KPI Creation)**

## **Exploratory Data Analysis (EDA)**

## **Insights Summary**

## **Business Recommendation**